In [1]:
import os
import pandas as pd
from tqdm import tqdm
import gzip
from os import listdir
from os.path import isfile, join
from chardet import detect
import re
import numpy as np
import shutil

import openpyxl as xl
import json
from collections import OrderedDict
import soundfile as sf
import librosa

In [2]:
### metadata.json 만들 db 선택 (ex:NC_PTTS)

In [5]:
input_value = input()

 NC_PTTS


In [6]:
data_path = './'+input_value+'/BnS/'
files = listdir(data_path)

In [7]:
del_file = []
startwith_str = []

for i in files:
    if os.path.isdir(data_path+i):
        del_file.append(i)
        if re.search('^\w', i):
            startwith_str.append(i)

In [8]:
for i in tqdm(startwith_str, mininterval=1):
    
    # 화자별 wav 파일리스트 생성
    wav_files = listdir(data_path+i+'/v0/wavs')
    
    # 음원파일의 duration, sample_rate, 파일명 경로, 스피커 구하기
    duration_li = []
    rate_li =[]
    fname_li = []
    spk_li = []
    
    for j in wav_files:
        
        # duration, sample_rate
        f = sf.SoundFile(data_path+i+'/v0/wavs/'+j)
        rate = f.samplerate
        dur = round((len(f) / f.samplerate),2)
        rate_li.append(rate)
        duration_li.append(dur)
    
        # 파일경로, speaker
        def clean_text(text):
            pattern = '(\.wav)$'
            text = re.sub(pattern=pattern, repl='', string=text)
            text = text.strip()
            text = " ".join(text.split())
            return text
        
        k = input_value+'/'+i+'/item/'+j
        k = clean_text(k)
        q = k.split('/')
        fname_li.append(k)
        spk_li.append(q[0]+'/'+q[1])
    
    # 텍스트 내용 불러오기
    txt_path = data_path+i+'/_scripts/ncbs_transcription.xlsx'
    wb = xl.load_workbook(txt_path)
    ws = wb.active
    
    txt_li = []
    for r in ws.rows:
        txt_li.append(r[1].value)
    
    # df로 병합
    df = pd.DataFrame(fname_li, columns =['fname'])
    df.index += 1
    df['text'] = txt_li
    df['duration'] = duration_li
    df['speaker'] = spk_li
    df['sample_rate'] = rate_li
    
    # 데이터프레임 저장 (csv, xlsx)
    if not os.path.exists('./'+input_value+'_meta'):
        os.makedirs('./'+input_value+'_meta')

    df.to_csv('./'+input_value+'_meta/metadata_'+i+'.csv', encoding='utf-8-sig', header=None)
    df.to_excel('./'+input_value+'_meta/metadata_'+i+'.xlsx', encoding='utf-8-sig', header=None)
    
    # json 생성
    xlsx_path = './'+input_value+'_meta/metadata_'+i+'.xlsx'
    wb_2 = xl.load_workbook(xlsx_path)
    ws_2 = wb_2.active
    dic = OrderedDict()
    
    for z in ws_2.rows:
        dic[z[0].value] = {'fname':z[1].value,
                           'text':z[2].value,
                           'duration':z[3].value,
                           'speaker':z[4].value,
                           'sample_rate':z[5].value}
    
    with open('./'+input_value+'_meta/metadata_'+i+'.json', 'w', encoding="utf-8") as make_file:
        json.dump(dic, make_file, ensure_ascii=False, indent="\t")

100%|██████████████████████████████████████████████████████████████████████████████████| 10/10 [00:03<00:00,  3.17it/s]
